# Multi-XRF Using MVR

In [61]:
# If you have installation questions, please reach out
import pandas as pd # data storage
import numpy as np  # math and stuff

import sklearn  
import datetime

from sklearn import linear_model

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error, max_error, mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [62]:
import defaults
from defaults import framecleaner, splitterz

# Dataframes

In [63]:
df0  = pd.read_csv('../../core_to_wl_merge/OS0_Merged_dataset_imputed_08_23_2021.csv')

df1 = pd.read_csv('../../core_to_wl_merge/OS1_Merged_dataset_imputed_08_23_2021.csv')

df2 = pd.read_csv('../../core_to_wl_merge/OS2_Merged_dataset_imputed_08_23_2021.csv')

In [64]:
param_dict ={
    "dataset": ['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE', 'RESD', 'PHIN', 'PHID', 'GR_smooth',  'PE_smooth', 'Ti', 'Mg', 'Si', 'Al', 'Ca'],
    "inputs": ['CAL', 'GR', 'DT', 'SP', 'DENS', 'PE', 'RESD', 'PHIN', 'PHID', 'GR_smooth',  'PE_smooth'],
    "target": ['Ti', 'Mg', 'Si', 'Al', 'Ca']}

In [65]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num', 'sample' ], axis=1)

df = df[df.Al >= 0]

NameError: name 'df' is not defined

In [ ]:
dataset = df[[
  'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
  'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth', 
  'Al', 'Ti', 'Si', 'Mg', 'Ca']]

dataset.replace('NaN',np.nan, regex=True, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
X = dataset[[ 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth']]

Y = dataset[['Al', 'Ti', 'Si', 'Mg', 'Ca']]

In [ ]:
seed = 42 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y.values, test_size=test_size)

In [ ]:
def mvr_multi_xrf(X_train, X_test, y_train, y_test, OS='os'):
  regr = linear_model.LinearRegression()
  regr.fit(X_train, y_train)

  print('Intercept: \n', regr.intercept_)
  print('Coefficients: \n', regr.coef_)

  preds = regr.predict(X_test)

  rmse5 = mean_squared_error(y_test, preds, squared=False)
  #print("Mean Squared Error: %f" % (rmse5))
  MAE2 = median_absolute_error(y_test, preds)
  #print("Median Abs Error: %f" % (MAE2))

  x = datetime.datetime.now()

  d = {'target': ['Multi-XRF'],
      'Offset':[OS],
     'RMSE': [rmse5],
     'MAE': [MAE2], 
     'day': [x.day], 
     'month':[x.month], 
     'year':[x.year],
     'model':['MVR'],
     'version':[sklearn.__version__]}

  results = pd.DataFrame(data=d)

  return results

# Offset 0

In [ ]:
df0 = df0[df0.Al >= 0]

#Create the dataset
X, Y_array = framecleaner(df0, param_dict['dataset'], param_dict['inputs'], param_dict['target'] )

#Split the dataset
X_train, X_test, y_train, y_test = splitterz(X.values, Y_array)

df_OS0 = mvr_multi_xrf(X_train, X_test, y_train, y_test, OS='OS0')
df_OS0

Intercept: 
 [ 0.1960738   1.45355955 25.23525796  1.71542907  8.04933779]
Coefficients: 
 [[-4.86508006e-03 -3.21947500e-01 -4.99975220e-02 -3.29722556e-02
  -3.25234857e-02 -6.72057669e-02 -4.90927598e-03 -1.53022965e-02
  -1.53022965e-02  5.23487210e-01  9.49914020e-02]
 [-2.98669860e-01 -2.06609381e+00 -1.76407592e-01 -3.64718883e-01
  -5.33237125e-01 -1.91039153e-01 -3.88752377e-03 -5.94015383e-01
  -5.94015383e-01  3.25649624e+00  1.10435792e-01]
 [ 2.21606647e+00  9.95095140e+00  1.87307477e+00 -1.89624870e+00
   1.09045652e+01  1.14420372e-01  3.29337179e-01  5.78577863e+00
   5.78577863e+00 -1.77969549e+01  2.84378061e+00]
 [ 6.32941561e-01 -5.11980812e+00 -4.03389526e-01 -3.25367534e-01
   1.22671930e+00 -9.67807511e-01 -8.84068820e-02  4.83048270e-01
   4.83048270e-01  8.21491922e+00  1.16480434e+00]
 [-2.09852349e+00  2.00517150e-01 -8.25288624e-01  6.94845435e-02
  -4.43486224e+00  1.02289839e+00 -2.05179041e-02 -3.00825121e+00
  -3.00825121e+00  1.38663574e+00 -3.15969235

C:\Users\tmartin\.conda\envs\lewisml\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,target,Offset,RMSE,MAE,day,month,year,model,version
0,Multi-XRF,OS0,1.382755,0.705145,9,9,2021,MVR,0.24.2


# Offset 1

In [ ]:
#Create the dataset
X1, Y1_array = framecleaner(df1, param_dict['dataset'], param_dict['inputs'], param_dict['target'] )

#Split the dataset
X1_train, X1_test, y1_train, y1_test = splitterz(X1.values, Y1_array)

df_OS1 = mvr_multi_xrf(X1_train, X1_test, y1_train, y1_test, OS='OS1')
df_OS1

Intercept: 
 [ 0.0773774   0.4705955  31.43677596 -0.63053071  5.85375398]
Coefficients: 
 [[-3.39537520e-03  1.66000169e-01  8.35310767e-03 -9.31643975e-03
   4.32194164e-02 -5.29345844e-02 -5.64483443e-03  2.25106199e-03
   2.25106199e-03  3.90305715e-02  1.35079724e-01]
 [-1.11568914e-01 -1.49800892e-01  2.22820233e-01 -1.77785193e-01
   7.56980042e-02 -1.13272796e-01 -2.98772149e-03 -4.11659394e-01
  -4.11659394e-01  1.21250943e+00  5.12045531e-01]
 [ 1.89238551e+00 -2.20467678e+00 -6.55556370e-01 -3.15628893e+00
   6.90141159e+00 -1.32353629e+00  3.00097774e-01  4.99748037e+00
   4.99748037e+00 -5.25488802e+00  6.55927365e-01]
 [ 8.44890103e-01  2.69246082e+00  6.12680858e-01  1.44631171e-01
   2.65768568e+00 -7.62693449e-01 -1.15824454e-01  9.24408504e-01
   9.24408504e-01  9.58651948e-02  2.43892505e+00]
 [-1.39259151e+00 -3.24113532e+00 -3.35877968e-01  4.72835298e-01
  -3.03077437e+00  1.41932045e+00  3.32751866e-02 -2.67216091e+00
  -2.67216091e+00  4.23684912e+00 -2.22118195

C:\Users\tmartin\.conda\envs\lewisml\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,target,Offset,RMSE,MAE,day,month,year,model,version
0,Multi-XRF,OS1,1.309912,0.623657,9,9,2021,MVR,0.24.2


# Offset 2

In [ ]:
#Create the dataset
X2, Y2_array = framecleaner(df2, param_dict['dataset'], param_dict['inputs'], param_dict['target'] )

#Split the dataset
X2_train, X2_test, y2_train, y2_test = splitterz(X2.values, Y2_array)

df_OS2 = mvr_multi_xrf(X2_train, X2_test, y2_train, y2_test, OS='OS2')
df_OS2

Intercept: 
 [ 0.06263845  0.42155584 31.67211095 -0.71235473  6.37928336]
Coefficients: 
 [[ 2.21285475e-02  3.50344700e-01  7.43621817e-03 -1.87763162e-02
   6.06077670e-02 -2.07454134e-02 -5.74174475e-03  7.76178370e-03
   7.76178370e-03 -1.38822573e-01  9.27618839e-02]
 [-1.88743802e-02  1.12384848e+00  2.13909831e-01 -2.05968209e-01
   1.28615286e-01 -1.47511785e-01  1.22984559e-03 -4.09016188e-01
  -4.09016188e-01  4.34779721e-02  4.32724418e-01]
 [ 1.19053388e+00 -9.65801234e+00 -6.32553055e-01 -3.07623732e+00
   6.54624243e+00 -1.39200854e+00  2.66323778e-01  4.94723045e+00
   4.94723045e+00  2.00083982e+00  1.51351827e+00]
 [ 1.14434856e+00  6.73242021e+00  5.00586089e-01  3.65124979e-02
   2.80894022e+00 -2.91344073e-01 -1.10629816e-01  9.65988058e-01
   9.65988058e-01 -3.72235785e+00  1.68468899e+00]
 [-1.64443526e+00 -3.40346525e+00 -4.27146051e-01  6.25155068e-01
  -3.48472006e+00  1.60534386e+00  6.38536955e-02 -2.86564032e+00
  -2.86564032e+00  4.57888726e+00 -2.93023507

C:\Users\tmartin\.conda\envs\lewisml\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,target,Offset,RMSE,MAE,day,month,year,model,version
0,Multi-XRF,OS2,1.094886,0.566031,9,9,2021,MVR,0.24.2


# Combine Datasets

In [ ]:
frames = [df_OS0, df_OS1, df_OS2]
results = pd.concat(frames)
results

,target,Offset,RMSE,MAE,day,month,year,model,version
0,Multi-XRF,OS0,1.382755,0.705145,9,9,2021,MVR,0.24.2
0,Multi-XRF,OS1,1.309912,0.623657,9,9,2021,MVR,0.24.2
0,Multi-XRF,OS2,1.094886,0.566031,9,9,2021,MVR,0.24.2
